In [1]:
using JSON
using Random
using Distributions
include("/home/peters/code/mapinator/estimation/functions.jl")
YEAR_INTERVAL = 2003:2023  # change this to select the years of data to include in the estimation
NUM = 5 # variable set it here
NUMSINKS = 6 #4 sinks are assumed in some of the code
sinks = ["Other Academic","Government","Private Sector","Teaching Universities","Postdoc", "Sessionals","Column Totals"]

7-element Vector{String}:
 "Other Academic"
 "Government"
 "Private Sector"
 "Teaching Universities"
 "Postdoc"
 "Sessionals"
 "Column Totals"

bucket_extract

In [2]:
 academic_builder, sink_builder, academic, acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer = functions.process_data(YEAR_INTERVAL);

Debug: 14876 returned from api
Debug: 7272 academic placements
Debug: 7604 sinks
Debug: 14876 in all


In [3]:
println(length(academic_builder), " total assistant professor outcomes")
    println(length(sink_builder), " other outcomes")
    sample_size = length(academic_builder)+length(sink_builder)
    println(sample_size, " total outcomes")
println(length(acd_sink), " Other academic institutions")
println(length(gov_sink), " Government institutions")
println(length(pri_sink), " Private Sector institutions")
println(length(tch_sink), " Teaching institutions")
println(length(postdoc), " Institutions hiring postdocs")
println(length(lecturer), " Institutions hiring sessionals")


7272 total assistant professor outcomes
7604 other outcomes
14876 total outcomes
552 Other academic institutions
156 Government institutions
196 Private Sector institutions
627 Teaching institutions
515 Institutions hiring postdocs
366 Institutions hiring sessionals


In [4]:
institutions, out = functions.create_adjacency_matrix(academic_builder, sink_builder, academic,acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer);

In [5]:
est_obj, est_alloc = functions.doit(out, collect(academic),collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink), collect(postdoc), collect(lecturer),institutions, NUM, NUMSINKS);

In [6]:
est_mat, est_count = functions.bucket_extract(est_alloc, out, NUM, NUMSINKS)
M = est_mat

11×5 Matrix{Int32}:
 348  115   14  675   35
 808  386   21  778   92
  33   44   52   13   52
 106   44    2  703   12
 439  254   23  276  151
 420  388  191  416  297
 457  254   38  545  104
 402  223   27  673   75
 578  515  113  380  210
 617  584  195  559  336
 221  188   94  141  159

In [7]:
t_table, t_allocation = functions.tiered_allocation(M, est_count, institutions, est_alloc, NUM, NUMSINKS);
t_table

11×5 Matrix{Int32}:
 703  106   44   12    2
 675  348  115   35   14
 778  808  386   92   21
 276  439  254  151   23
  13   33   44   52   52
 416  420  388  297  191
 545  457  254  104   38
 673  402  223   75   27
 380  578  515  210  113
 559  617  584  336  195
 141  221  188  159   94

In [8]:
table_latex = functions.nice_table(t_table, NUM, NUMSINKS, sinks)
table_latex

┌───────────────────────┬────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Tier 5 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │    703 │    106 │     44 │     12 │      2 │        867 │
│                Tier 2 │    675 │    348 │    115 │     35 │     14 │       1187 │
│                Tier 3 │    778 │    808 │    386 │     92 │     21 │       2085 │
│                Tier 4 │    276 │    439 │    254 │    151 │     23 │       1143 │
│                Tier 5 │     13 │     33 │     44 │     52 │     52 │        194 │
│        Other Academic │    416 │    420 │    388 │    297 │    191 │       1712 │
│            Government │    545 │    457 │    254 │    104 │     38 │       1398 │
│        Private Sector │    673 │    402 │    223 │     75 │     27 │       1400 │
│ Teaching Universities │    380 │    578 │    515 │    210 │    113 │      

In [9]:
for j in 1:NUM
    println("Tier $j:")
    for (i, type) in enumerate(t_allocation)
        if type == j
            println("  ", institutions[i])
        end
    end
println()
    end

Tier 1:
  University of Wisconsin, Madison
  Stanford University
  Duke University
  University of Maryland
  Northwestern University
  London School of Economics and Political Science
  Harvard University
  Yale University
  University of Michigan
  Columbia University
  Massachusetts Institute of Technology
  New York University
  Princeton University
  University of Chicago
  University of Pennsylvania
  University of California, Berkeley
  University of California Los Angeles (UCLA)

Tier 2:
  Boston College
  University of Warwick
  National University of Singapore
  Bocconi University
  Ohio State University
  University of Cambridge
  University of Texas at Austin
  Vanderbilt University
  Erasmus University Rotterdam
  Johns Hopkins University
  University College London
  University of Bonn
  Arizona State University
  University of Rochester
  Toulouse School of Economics
  University of Virginia
  Cornell University
  University of British Columbia
  Maastricht University
  